In [1]:
import os

train_data_path = "./data/train_essays.csv"
supplement_data_dir = "./data/archive/"
supplement_data_files = [  os.path.join(supplement_data_dir,f)  
                          for f in os.listdir(supplement_data_dir)
                          if(f.endswith('.csv'))]

In [2]:
def set_proxy():
    import os
    cache_dir = "/home/tx/workspace/cache"  # 替换为你想要的缓存目录的路径
    os.environ['TOKENIZERS_PARALLELISM'] = 'true'
    # 代理
    os.environ['http_proxy'] = 'http://127.0.0.1:7890'
    os.environ['https_proxy'] = 'http://127.0.0.1:7890'
    os.environ['no_proxy'] = '127.0.0.1,localhost'
    os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
    os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
    os.environ['NO_PROXY'] = '127.0.0.1,localhost'
    return
set_proxy()

In [3]:
!curl huggingface.co

<html>
<head><title>301 Moved Permanently</title></head>
<body>
<center><h1>301 Moved Permanently</h1></center>
<hr><center>CloudFront</center>
</body>
</html>


In [4]:
import pandas as pd

supplement_train_data = pd.concat([
    pd.read_csv(f)
    for f in supplement_data_files
])
train_data=pd.read_csv(train_data_path)

In [5]:
supplement_train_data

,text,label,source,fold,essay_id,prompt
0,There are alot reasons to keep our the despise...,0,persuade_corpus,2,NaN,NaN
1,Driving smart cars that drive by themself has ...,0,persuade_corpus,4,NaN,NaN
2,"Dear Principal,\n\nI believe that students at ...",0,persuade_corpus,0,NaN,NaN
3,"Dear Principal,\n\nCommunity service should no...",0,persuade_corpus,0,NaN,NaN
4,My argument for the development of the driverl...,0,persuade_corpus,3,NaN,NaN
...,...,...,...,...,...,...
39780,"""Oh man I didn't make the soccer team!"", yelle...",0,persuade_corpus,4,F7341069C4A4,NaN
39781,I believe that using this technology could be ...,0,persuade_corpus,2,AFE6E553DAC2,NaN
39782,The Face on Mars is a fascinating phenomenon t...,1,falcon_180b_v1,2,falcon_180b_v1_600,You have read the article 'Unmasking the Face ...
39783,Texting & Driving\n\nUsing your phone while dr...,0,persuade_corpus,2,A5F84C104693,NaN


In [6]:
train_data.columns

Index(['id', 'prompt_id', 'text', 'generated'], dtype='object')

In [7]:
import gc
#del train_data_re
gc.collect()
train_data_re = pd.DataFrame(train_data[['text','generated']])


In [8]:
train_data

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...,...
1373,fe6ff9a5,1,There has been a fuss about the Elector Colleg...,0
1374,ff669174,0,Limiting car usage has many advantages. Such a...,0
1375,ffa247e0,0,There's a new trend that has been developing f...,0
1376,ffc237e9,0,As we all know cars are a big part of our soci...,0


## build base_model

In [9]:
cache_dir = "/home/tx/workspace/cache"  # 替换为你想要的缓存目录的路径
from transformers import AutoModelForCausalLM,AutoTokenizer
from transformers import AutoConfig, LlamaConfig 
# from llava.model.language_model.llava_llama import LlavaLlamaForCausalLM
# class LlavaConfig(LlamaConfig):
#     model_type = "llava"
# AutoConfig.register("llava", LlavaConfig)
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
#model_name = "ChocoWu/nextgpt_7b_tiva_v0"#"liuhaotian/llava-v1.5-7b"
model = AutoModelForCausalLM.from_pretrained(model_name,cache_dir=cache_dir,trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=cache_dir)
# tokenizer.add_tokens(["[PAD]"])

/home/tx/.conda/envs/llava/lib/python3.11/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


## dataset

In [ ]:
train_data_re

,text,generated
0,Cars. Cars have been around since they became ...,0
1,Transportation is a large necessity in most co...,0
2,"""America's love affair with it's vehicles seem...",0
3,How often do you ride in a car? Do you drive a...,0
4,Cars are a wonderful thing. They are perhaps o...,0
...,...,...
1373,There has been a fuss about the Elector Colleg...,0
1374,Limiting car usage has many advantages. Such a...,0
1375,There's a new trend that has been developing f...,0
1376,As we all know cars are a big part of our soci...,0


In [ ]:
tokenizer.eos_token_id

2

In [ ]:
from torch import Tensor
from torch.utils.data import TensorDataset,DataLoader,RandomSampler

class myDataset(TensorDataset):

    def __init__(self, datalist,max_length=256,tokenizer=None) -> None:
        super(myDataset,self).__init__()

        if(isinstance(datalist,pd.DataFrame)):
            self.datalist = datalist.to_dict(orient='list')
        elif(isinstance(datalist,dict)):
            self.datalist = datalist
        else:
            raise Exception("错误输入类型")
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.datalist)
    
    def __getitem__(self, index):
        text = self.datalist['text'][index]  
        inputs = self.tokenizer.encode(text)
        act_len = len(inputs)
        while(len(inputs) < self.max_length):
            inputs.append(self.tokenizer.eos_token_id)
        if(len(inputs) > self.max_length):
            inputs = inputs[:self.max_length]
        att_mask = [1] * min(act_len,len(inputs))
        att_mask = att_mask if len(att_mask) >= len(inputs) else att_mask + [0] * (len(inputs) - len(att_mask))
        if('generated' in self.datalist):
            label =  self.datalist['generated'][index]
            return  {'input_ids':inputs, 'label':label,'att_mask':att_mask}
        return {'input_ids':inputs,'att_mask':att_mask}

        
dataset = myDataset(datalist=train_data_re,max_length=1024,tokenizer=tokenizer) 




In [ ]:
dataset[0]

{'input_ids': [1,
  23508,
  28723,
  23508,
  506,
  750,
  1401,
  1854,
  590,
  3246,
  8376,
  297,
  272,
  28705,
  28740,
  28774,
  28734,
  28734,
  28713,
  28725,
  739,
  6808,
  12101,
  3859,
  304,
  4429,
  272,
  907,
  8871,
  28738,
  28723,
  23508,
  506,
  4226,
  264,
  3014,
  3905,
  297,
  813,
  1012,
  1370,
  4621,
  1854,
  868,
  28723,
  1092,
  1055,
  28725,
  905,
  460,
  5615,
  298,
  2996,
  513,
  27931,
  1253,
  11753,
  682,
  347,
  264,
  1179,
  1970,
  28723,
  1791,
  528,
  28725,
  27931,
  272,
  938,
  302,
  8300,
  1659,
  347,
  264,
  1179,
  1970,
  298,
  511,
  28723,
  13,
  13,
  657,
  737,
  3209,
  302,
  456,
  28725,
  5447,
  28725,
  345,
  657,
  5567,
  5078,
  5354,
  28725,
  5372,
  3187,
  274,
  1418,
  11528,
  23508,
  862,
  486,
  11289,
  8550,
  308,
  6019,
  4605,
  28725,
  910,
  4607,
  598,
  3429,
  460,
  272,
  12893,
  338,
  10002,
  302,
  24832,
  1816,
  28725,
  970,
  4986,
  875,
  6812,


In [ ]:
random_sampler = RandomSampler(dataset)

In [ ]:

dataloader_random = DataLoader(dataset, batch_size=2, sampler=random_sampler)


In [ ]:
import torch

In [ ]:
model(input_ids = torch.LongTensor([dataset[0]['input_ids']]),attention_mask=torch.LongTensor([dataset[0]['att_mask']]))

TypeError: MistralForCausalLM.forward() got an unexpected keyword argument 'attion_mask'

## model definition

In [ ]:
import torch 

class ai_classifier(torch.nn.Module):

    def __init__(self, base_model) -> None:
        super(ai_classifier,self).__init__()

        self.base_model = base_model

    def forward(self,inputs):
        self.base_model()

    def forward(self,input,tokenizer):
        pass



In [1]:
model

NameError: name 'model' is not defined